# EmergencyZIP AI

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip uninstall -qqy jupyterlab  # Remove unused packages from Kaggle's base image that conflict
!pip install -U -q "google-genai==1.7.0"

In [ ]:
from google import genai
from google.genai import types

from IPython.display import HTML, Markdown, display

In [ ]:
from google.api_core import retry


is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

In [ ]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

In [ ]:
client = genai.Client(api_key=GOOGLE_API_KEY)

In [ ]:
# When using as interactive AI, uncomment this whole section
"""
medical_emergency = input("Hello, what medical emergency are you dealing with?")
zipcode = int(input("What zipcode are you currently living now?"))

medical_request = {
    'Zipcode': zipcode, 
    'Medical Emergency': medical_emergency
}
medical_request_str = str(medical_request)
medical_request_str
"""

# When saving to GitHub, uncomment this whole section 
medical_emergency = 'broken arm'
zipcode = 77035

medical_request = {
    'Zipcode': zipcode, 
    'Medical Emergency': medical_emergency
}
medical_request_str = str(medical_request)
medical_request_str

In [ ]:
import typing_extensions as typing

class MedicalRequest(typing.TypedDict): 
    zipcode: int
    medical_emergency: str

json_response = client.models.generate_content(
    model='gemini-2.0-flash', 
    config=types.GenerateContentConfig(
        temperature=0.1, 
        response_mime_type="application/json", 
        response_schema=MedicalRequest
    ), 
    contents=medical_request_str
)

json_form = json_response.text

print(json_form)

In [ ]:
few_shot_prompt = """Parse JSON into a string request and only return output: 

EXAMPLE: 
{
"zipcode": 78664, 
"medical_emergency": "heart attack" 
}
"Patient currently living in zipcode 78664 wants to seek medical attention for heart attack."

EXAMPLE: 
{
"zipcode": 77055, 
"medical_emergency": "broken leg" 
}
"Patient currently living in zipcode 77055 wants to seek medical attention for broken leg."

EXAMPLE: 
{
"zipcode": 78681, 
"medical_emergency": "dog bite" 
}
"Patient currently living in zipcode 78681 wants to seek medical attention for dog bite."
"""

model_config = types.GenerateContentConfig(
    temperature=0.1, 
    top_p=1, 
    max_output_tokens=500
)

few_shot_response = client.models.generate_content(
    model='gemini-2.0-flash', 
    config=model_config,
    contents=[few_shot_prompt, json_form]
)

str_request = few_shot_response.text
print(str_request)

In [ ]:
prompt_config = types.GenerateContentConfig(
    temperature=0.1, 
    top_p=1.0, 
    max_output_tokens=250
)

generate_prompt_prompt = "Generate precise prompt that asks for nearest medical centers for this request: " + str_request

prompt_response = client.models.generate_content(
    model='gemini-2.0-flash', 
    config=prompt_config, 
    contents= generate_prompt_prompt
)

prompt = prompt_response.text
print(prompt)

In [ ]:
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
)

response = client.models.generate_content(
    model='gemini-2.0-flash', 
    contents=prompt, 
    config=config_with_search,
)

rc = response.candidates[0]

Markdown(rc.content.parts[0].text)